Vector AutoRegression:

Multiple equation model for joint determination of two or more variables

• One of the most commonly used models for applied macroeconometric analysis and forecasting in central banks

• In general: k variables 

• An equation for each variable


• There is one error per equation. – The errors are (typically) correlated.



## Motivation

### Unrestricted VAR

• An unrestricted VAR includes all variables in each equation

• A restricted VAR might include some variables in one equation, other variables in another equation

• Old‐fashioned macroeconomic models (so‐called simultaneous equations models of the 1950s, 1960s, 1970s) were essentially restricted VARs – The restrictions and specifications were derived from simplistic macro theory, e.g. Keynesian consumption functions, investment equations, etc.



### Sims and VARs

• Sims argued that the conventional models were restricted VARs, and the restrictions had no substantive justification   

   – Based on incomplete and/or non‐rigorous theory, or intuition

• Sims argued that economists should instead use unrestricted models, e.g. VARs

• He proposed a set of tools for use and evaluation of VARs in practice.

http://www.ssc.wisc.edu/~bhansen/390/


### A VAR is said to be stationary if 

- (1) the expected value of the dependent variables do not depend on time and 

- (2) the covariance matrix of a dependent variable and its lags depends only on the time lapsed and not on the reference period. 

In other words, a VAR system is stationary if the mean and variance of its variables are measurable and do not depend on time.

www.stata.com/manuals13/tsvarstable.pd

A VAR is stationary if all the roots of the characteristic polynomial lie outside the unit circle. EViews tests for stationarity by inverting these roots and plotting them relative to the unit circle. If an inverse root lies outside of the circle, then either some or all variables in the VAR(p)-process are integrated of order one, i.e., I(1). It might be the case, that cointegration between the variables does exist. This instance can then be better analyzed in the context of a VECM.

### reduced-form VARs and structural VARs


In the absence of exogenous variables, the disturbance variance-covariance matrix Σ contains all relevant information about contemporaneous correlation among the variables in yt. VARs may be **reduced-form VARs**, which do not account for this contemporaneous correlation.

they may be **structural VARs**, where theory is used to place restrictions on the contemporaneous correlations.

Stata has a complete suite of commands for fitting and forecasting vector autoregressive (VAR) models and structural vector autoregressive (SVAR) models. Its capabilities include estimating and interpreting impulse response functions (IRFs), dynamic multipliers, and forecast error vector decompositions (FEVDs).




Subsidiary commands allow you to check the stability condition of VAR or SVAR estimates; 

- to compute lag-order selection statistics for VARs; 

- to perform pairwise Granger causality tests for VAR estimates; 

- and to test for residual autocorrelation and normality in the disturbances of VARs.

- Dynamic forecasts may be computed and graphed after VAR or SVAR estimation.

- The `varsoc` command allows you to select the appropriate lag order for the VAR; 


- command `varwle` computes Wald tests to determine whether certain lags can be excluded; 


- `varlmar` checks for autocorrelation in the disturbances; 


- and `varstable` checks whether the stability conditions needed to compute IRFs and FEVDs are satisfied.

Declare and estimate an unrestricted small monetary VAR for Canada, 

- in output gap, (difference between real gdp and gdp trend using hp filter)

- inflation, 

- monetary policy rate 

- and real exchange rate gap, 

over the sample 1994q1 - 2012q4, using 4 lags





In [ ]:

%matplotlib inline
import seaborn as sns
import pandas as pd
import ipystata

In [ ]:
can = pd.read_csv("table01.csv")

In [ ]:
can.head()

In [ ]:
can.rename(columns = {'Unnamed: 0': 'yq'}, inplace = True)

In [ ]:
can.columns

In [ ]:
can.set_index(['yq'], inplace=True)

In [ ]:
%%stata -d can
*close all

summarize

In [ ]:
%%stata -d can
gen quarter= quarterly(yq,"yq")
format quarter %tq
tsset quarter


In [ ]:
%%stata

gen loggdp = log(GDP)

In [ ]:
%%stata 
summarize



http://www.stata.com/manuals13/tstsfilterhp.pdf

In [ ]:
%%stata --graph
    
tsline LGDP

In [ ]:
%%stata -d can
gen quarter= quarterly(yq,"yq")
format quarter %tq
tsset quarter

tsfilter hp realgdp_hp = LGDP , smooth(1600) trend(realgdp_tr)

In [ ]:
%%stata -d can
gen quarter= quarterly(yq,"yq")
format quarter %tq
tsset quarter
tsfilter hp lgdp_hp = LGDP , smooth(1600) trend(lgdp_tr)

In [ ]:
%%stata 
summarize

In [ ]:
%%stata --graph
    
tsline lgdp_tr

In [ ]:
%%stata --graph
    
tsline lgdp_hp

let’s estimate  VAR for Canada , which should be similar to the original canada VAR  that it uses the RER gap.  Estimate a VAR from 1993q1 to 2012q4 using lgdp_gap, rer, infl and mpr as the endogenous variables. Include a constant in your specification and 4 lags.

Hint: In EViews You can do this by opening the VAR Object, then: View / Lag Structure / AR Roots Graph.

In [ ]:
%%stata


var LGDP_GAP  INFL  MPR  RER_GAP  if tin(,2012q4), lags(1/4)


and `varstable` checks whether the stability conditions needed to compute IRFs and FEVDs are satisfied.

In [ ]:
%%stata --graph
stable, graph

As we did in a previous question, let’s estimate another VAR for Canada (let’s call it canada1), which should be similar to the original canada VAR except that it uses the RER in levels, rather than the RER gap. In all other respects the VARs are similar. Estimate a VAR from 1993q1 to 2012q4 using lgdp_gap, rer, infl and mpr as the endogenous variables. Include a constant in your specification and 4 lags.

Hint: You can do this by opening the VAR Object, then: View / Lag Structure / AR Roots Graph.

In [ ]:
%%stata --graph

var LGDP_GAP  INFL  MPR  RER  if quarter >=tq(1993q1) & quarter <=tq(2012q4), lags(1/4)



In [ ]:
%%stata --graph
varstable, graph

The original canada VAR is stable because all of its inverse roots are inside of the unit circle, as shown in the EViews graph below:

On the other hand, the second VAR canada1 is not stable because one of its roots is approximately (very close to) 1, as can be seen in the picture below:

## Lag Specification Criteria

VARs are very densely parameterized and exhibit the “curse” of dimensionality. 

Suppose that we are estimating a quarterly VAR with 3 endogenous variables, a constant and 4 lags.

In  case n=3 and p=4. The total number of coefficients to be estimated is n*(1+n*p) thus 3*(1+3*4) = 39.

The number of parameters grows at a quadratic rate as more variables/lags of variables are added, using up an increasing amount of degrees of freedom. So using too many lags worsens the “over-fitting” problem and generally leads to poor quality estimates and bad forecasts. 


On the other hand, including too few lags can lead to residuals not being white noises (for example display autocorrelation), which is an issue in a stationary VAR.

When estimating a VAR, why is the selection of the appropriate lag length crucial? (Select all that apply.)

If the number of lags is too long, the model will use up too many degrees of freedom, leading to poor estimates. 

If the number of lags is not sufficient, the residuals from the estimation may not constitute individual white noises. 



In a k‐variable system, the number of coefficients in each equation is 1+kp 


– The total number is k(1+2p)=k+2kp


• How should p be selected? 


• Common approach: – Information criterion, primarily AIC


#### AIC and BIC for VAR Models 


where L is log‐likelihood from model 

• Select model with smallest AIC (or BIC)


We may also want to compute selection order criteria to gauge whether we have included sufficient lags in the VAR. Introducing too many lags wastes degrees of freedom, while too few lags leave the equations potentially misspecified and are likely to cause autocorrelation in the residuals. The `varsoc` command will produce selection order criteria, and highlight the optimal lag.

The `varsoc` command allows you to select the appropriate lag order for the VAR; 



In [ ]:
%%stata 



varsoc LGDP_GAP  INFL  MPR  RER_GAP, maxlag(8)

According to the SC and HQ criteria, 2 lags are sufficient. The AIC would suggest 3 lags. This would suggest that 2 (at most 3 lags) should be sufficient. We shall proceed with 2 lags and re-estimate the model in later questions.


command `varwle` computes Wald tests to determine whether certain lags can be excluded; 

## VAR Diagnostics: Residuals 







In [ ]:
%%stata 
varwle

`varlmar` checks for autocorrelation in the disturbances; 



In [ ]:
%%stata 
varlmar

## Interpretation


• It is difficult to interpret the large number of coefficients in the VAR model


• Main tools for interpretation – Impulse responses


• The impulse responses are the time‐paths of to y and x in response to shocks


• The errors may be correlated. 


• We “orthogonalize” them


### Orthogonalized Model

• The shocks u1 and u2 are uncorrelated 

• The ordering matters – The shock to y affects both y and x in period t 

  – The shock to x affects only x in period t


• The impulse responses are the time‐paths of to y and x in response to the shocks u1 and u2


• Imagine y=0 and x=0. Set u1=1. Trace the history of y and x

#### Impulse Responses by Recursion


• The impulse responses are these time‐paths of y and x due to the shocks u1 and u2


• They are found by this recursion formula 


• They are functions of the estimated VAR coefficients

• In a k‐variable system, there are k2 impulse response functions!


##### Stata Calculation

• Impulse response automatically calculated with varbasic command

• A kxk matrix of impulse response is created



In [ ]:
%%stata --graph

varbasic LGDP MPR 

#### Interpretation


• Labeled “Graphs by irfname, impulse variable, and response variable” – “Impulse variable” means the source of the shock – “Response variable” means the variable being affected


• Upper left: “varbasic, gdp, gdp” – Impact of a gdp shock on the time‐path of gdp


• Upper right: “varbasic, gdp, rate” – Impact of a gdp shock on the time‐path of interest rates


• Lower left: “varbasic, rate, gdp,” – Impact of an interest rate shock on the time‐path of gdp


• Lower fight: “varbasic, rate, rate” – Impact of an interest rate shock on the time‐path of interest rates


• The impulse response is graphed as a function of forward time periods

In [ ]:
%%stata --graph

varbasic LGDP_GAP  INFL  MPR  RER_GAP

## Scale

• The graphs are all created on the same scale, so difficult to read

• It may be better to create graphs separate for each impulse response

• This creates the impulse response for the impact of a gdp shock on the time‐path of interest rates

In [ ]:
%%stata --graph

irf graph oirf, impulse(LGDP_GAP) response(RER_GAP)

In [ ]:
%%stata --graph

irf graph oirf, impulse(RER_GAP) response(LGDP_GAP)

In [ ]:
%%stata --graph
var LGDP_GAP  INFL  MPR  RER_GAP 
irf graph oirf, impulse(LGDP_GAP) response(INFL)

In [ ]:
%%stata --graph

irf graph oirf, impulse(LGDP_GAP) response(MPR)

For IRFs to be computed, the VAR must be stable. The simple IRFs shown above have a drawback: they give the effect over time of a one-time unit increase to one of the shocks, holding all else constant. But to the extent the shocks are contemporaneously correlated, the other shocks cannot be held constant, and the VMA form of the VAR cannot have a causal interpretation.




## FEVDs


The forecast error variance decomposition (FEVD) measures the fraction of the forecast error variance of an endogenous variable that can be attributed to orthogonalized shocks to itself or to another endogenous variable.



As any of the VAR estimation commands save the estimated IRFs, OIRFs and FEVDs in an .irf file, you may examine the FEVDs with a graph command. These items may also be tabulated with the `irf tabl` and `irf ctable` commands. 


The latter command allows you to juxtapose tabulated values, such as the OIRF and FEVD for a particular pair of variables, while the `irf cgraph` command allows you to do the same for graphs. . 

    irf graph fevd, lstep(1)


In [ ]:
%%stata --graph


irf graph fevd, lstep(1)

After producing any graph in Stata, you may save it in Stata’s internal format using graph save filename. 

This will create a .gph file which may be accessed with graph use. The file contains all the information necessary to replicate the graph and modify its appearance. However, only Stata can read .gph files. 

If you want to reproduce the graph in a document, use the graph export filename.format command, where format is .eps or .pdf.

A common diagnostic from a VAR are the set of block F tests, or Granger causality tests, that consider whether each variable plays a significant role in each of the equations. 


These tests may help to establish a sensible `causal ordering`. They can be performed by vargranger:

In [ ]:
%%stata 

var LGDP_GAP  INFL  MPR  RER_GAP  if quarter >=tq(1993q1) & quarter <=tq(2012q4), lags(1/4)
vargranger

In [ ]:
%%stata 

var RER_GAP LGDP_GAP INFL  MPR    if quarter >=tq(1993q1) & quarter <=tq(2012q4), lags(1/4)

In [ ]:
%%stata  --graph

varbasic RER_GAP LGDP_GAP INFL  MPR    if quarter >=tq(1993q1) & quarter <=tq(2012q4)

In [ ]:
#https://courses.edx.org/courses/course-v1:IMFx+MFx+1T2017/courseware/c7a914d5671e453eace54746594f5d63/af626a308c1745b79e785578a8d2bb6c/






A VAR is said to be stationary if (1) the expected value of the dependent variables do not depend on time and (2) the covariance matrix of a dependent variable and its lags depends only on the time lapsed and not on the reference period. In other words, a VAR system is stationary if the mean and variance of its variables are measurable and do not depend on time.

A VAR is stationary if all the roots of the characteristic polynomial lie outside the unit circle. EViews tests for stationarity by inverting these roots and plotting them relative to the unit circle. If an inverse root lies outside of the circle, then either some or all variables in the VAR(p)-process are integrated of order one, i.e., I(1). It might be the case, that cointegration between the variables does exist. This instance can then be better analyzed in the context of a VECM.

In [ ]:
http://fmwww.bc.edu/EC-C/S2013/823/EC823.S2013.nn10.slides.pdf